In [8]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter


In [ ]:
# Chemins des fichiers et dossiers
proteome_dir_path = "data/Proteome_1011"  # Dossier contenant les fichiers de séquences protéiques
phenotype_data_path = "data/Finalset_223phenotypes_1011.csv"
copy_number_variation_path = "data/genesMatrix_CopyNumber.tab"

# Pour les phénotypes et les copy variations

In [ ]:
# Chargement des données phénotypiques
phenotype_df = pd.read_csv(phenotype_data_path)

# Chargement des données de variation du nombre de copies
cnv_df = pd.read_csv(copy_number_variation_path, sep="\t", index_col=0).T
cnv_df.index.name = "Standard_name"

# Correction des valeurs manquantes
cnv_df.fillna(0, inplace=True)

# Réindexation pour une compatibilité ultérieure
cnv_df.reset_index(inplace=True)

# Pour les mutations pour chaque proteine sur chaque yeast

In [ ]:
# Dossier contenant les fichiers FASTA
proteome_dir_path = "data/Proteome_1011"

# Liste pour stocker les données de mutation
mutations = []

# Parcourir chaque fichier dans le dossier des protéines
for filename in os.listdir(proteome_dir_path):
    if filename.endswith(".fasta"):
        filepath = os.path.join(proteome_dir_path, filename)
        protein_id = filename.split(".")[0]  # Nom de la protéine
        
        # Lecture des séquences du fichier FASTA
        sequences = {}
        with open(filepath, "r") as file:
            current_yeast = None
            for line in file:
                if line.startswith(">"):  # Identifiant de la levure
                    current_yeast = line.strip().split()[0][1:]  # Retirer le ">"
                    sequences[current_yeast] = ""
                else:
                    sequences[current_yeast] += line.strip()
        
        # Transposer les séquences pour analyser position par position
        yeast_ids = list(sequences.keys())
        transposed_positions = zip(*sequences.values())  # Colonne par colonne
        
        # Identifier les mutations par rapport à l'acide aminé majoritaire
        for idx, amino_acids in enumerate(transposed_positions, start=1):
            aa_counts = Counter(amino_acids)
            most_common_aa, _ = aa_counts.most_common(1)[0]  # Acide aminé majoritaire
            
            for yeast_id, aa in zip(yeast_ids, amino_acids):
                if aa != most_common_aa:  # Mutation détectée
                    mutations.append({
                        "Protein_ID": protein_id,
                        "Yeast_ID": yeast_id,
                        "Position": idx,
                        "Reference_AA": most_common_aa,
                        "Mutated_AA": aa
                    })

# Convertir les mutations en DataFrame
mutations_df = pd.DataFrame(mutations)

KeyboardInterrupt: 

In [13]:
# 4. Sauvegarder les datasets dans des fichiers CSV
mutations_output_path = "data/mutations_proteins_dataset.csv"
phenotype_output_path = "data/phenotype_dataset.csv"
cnv_output_path = "data/copy_number_variation_dataset.csv"

mutations_df.to_csv(mutations_output_path, index=False)
phenotype_df.to_csv(phenotype_output_path, index=False)
cnv_df.to_csv(cnv_output_path, index=False)

print(f"Mutations dataset saved to {mutations_output_path}")
print(f"Phenotype dataset saved to {phenotype_output_path}")
print(f"Copy number variation dataset saved to {cnv_output_path}")

Mutations dataset saved to data/mutations_proteins_dataset.csv
Phenotype dataset saved to data/phenotype_dataset.csv
Copy number variation dataset saved to data/copy_number_variation_dataset.csv
